In [1]:
import numpy as np
import pandas as pd
import cv2
import re
import os

# combine all data and clean

In [9]:
df_moviegenre = pd.read_csv('../files/movie-genre-from-its-poster/MovieGenre.csv', encoding='latin1')

df_movies = pd.read_csv('../files/the-movies-dataset/movies_metadata.csv', encoding='latin1')
df_credits = pd.read_csv('../files/the-movies-dataset/credits.csv', encoding='latin1')
df_keywords = pd.read_csv('../files/the-movies-dataset/keywords.csv', encoding='latin1')


In [8]:
# take only English movies and relaesed after 1980 
df_movies = df_movies[df_movies['original_language'] == 'en']
df_movies = df_movies.astype({'original_language': str, 'release_date': np.datetime64,
                             'id': np.int64})
df_movies = df_movies[df_movies['release_date'] >= '1980-01-01']

In [10]:
df_credits.duplicated(['id']).sum()

44

In [12]:
df_moviegenre.imdbId

0        114709
1        113497
2        113228
3        114885
4        113041
          ...  
40103     83168
40104     82875
40105    815258
40106     79142
40107     70710
Name: imdbId, Length: 40108, dtype: int64

In [13]:
df_credits = df_credits.drop_duplicates(subset=['id'])
df_keywords = df_keywords.drop_duplicates(subset=['id'])

df_moviegenre.rename({'imdbId': 'imdb_id'}, axis='columns', inplace = True)
df_moviegenre = df_moviegenre.drop_duplicates(subset = 'imdb_id')

In [23]:
# df_movies['id'] = df_movies['id'].astype(str)

In [28]:
# df_credits['id'] = df_credits['id'].astype(str)
# df_keywords['id'] = df_keywords['id'].astype(str)

In [27]:
df = df_movies.merge(df_credits, how='inner', on='id')

In [29]:
df = df.merge(df_keywords, on='id')

In [30]:
df = df.astype({'imdb_id': str})

In [32]:
df['imdb_id'] = df['imdb_id'].apply(lambda s: re.sub('[a-zA-Z]+', '', s))

In [33]:
df = df[df['imdb_id'] != ''].astype({'imdb_id': np.int64})

In [34]:
df = df.merge(df_moviegenre, on='imdb_id')

In [35]:
df = df.drop_duplicates(subset = 'imdb_id')

In [39]:
df.to_csv('../movies_all.csv', index=False)

## end combining

In [101]:
df = pd.read_csv('../movies.csv')

In [103]:
df[df['imdb_id'] == 114709]

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,vote_average,vote_count,cast,crew,keywords,Imdb Link,Title,IMDB Score,Genre,Poster
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,...,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...


In [156]:
imdb_ids = os.listdir('../posters/')
ids = [re.sub('.[a-zA-Z]+', '', s) for s in imdb_ids]
existing_posters_ids = np.intersect1d(ids, np.array(df.imdb_id))
existing_posters_ids = existing_posters_ids.astype(np.int64)

df = df.drop(columns = ['index'])

df = df[df['imdb_id'].apply(lambda x: x in existing_posters_ids)]
df = df.reset_index()

In [157]:
df.shape

(20335, 33)

In [158]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")
num_faces = np.zeros(df.shape[0])

brightness = np.zeros(df.shape[0])
saturation = np.zeros(df.shape[0])
hue = np.zeros(df.shape[0])
brightness_sd = np.zeros(df.shape[0])
saturation_sd = np.zeros(df.shape[0])
hue_sd = np.zeros(df.shape[0])

blue = np.zeros(df.shape[0])
blue_sd = np.zeros(df.shape[0])
green = np.zeros(df.shape[0])
green_sd = np.zeros(df.shape[0])
red = np.zeros(df.shape[0])
red_sd = np.zeros(df.shape[0])

for i in range(df.shape[0]):
#     print(df['imdb_id'][i])
    image = cv2.imread('../posters/' + str(df['imdb_id'][i]) + '.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, minSize=(100, 100))
    num_faces[i] = len(faces)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness[i] = np.mean(hsv[:,:,2])
    saturation[i] = np.mean(hsv[:,:,1])
    hue[i] = np.mean(hsv[:,:,0])
    brightness_sd[i] = np.std(hsv[:,:,2])
    saturation_sd[i] = np.std(hsv[:,:,1])
    hue_sd[i] = np.std(hsv[:,:,0])

    blue[i] = np.mean(image[:,:,0])
    green[i] = np.mean(image[:,:,1])
    red[i] = np.mean(image[:,:,2])
    blue_sd[i] = np.std(image[:,:,0])
    green_sd[i] = np.std(image[:,:,1])
    red_sd[i] = np.std(image[:,:,2])

    if i % 1000 == 0:
        print("iter", i)


114709
iter 0
113497
113228
114885
113041
113277
114319
112302
114576
113189
112346
112896
112453
113987
112760
112641
114388
113101
112281
113845
113161
112722
112401
114168
113627
114057
114011
114117
112792
114746
112431
112637
112818
112286
113442
112697
112749
114279
112819
114272
113855
114681
113347
114369
114148
114916
114814
109950
113819
112499
113158
113541
113321
113419
116260
113862
116126
118002
115683
116839
114753
113118
116367
113010
113537
115644
115676
114367
113973
112744
116731
114660
114039
113612
112365
118158
116151
115697
113972
113490
117002
114825
115639
115759
113403
111173
113283
115907
115734
117102
118040
116483
112579
117110
112646
113276
112573
115645
113774
112556
112373
115033
116606
114536
117427
106473
103859
116324
110647
110217
114131
112364
113451
114500
116130
113125
113756
113952
118055
115685
113200
112585
112442
112342
112461
112427
109093
112384
114287
112288
112462
112495
109370
112642
112688
112715
112740
109508
112851
112857
112864
112887

119527
119978
118749
90329
119365
120201
118901
107282
118836
120148
119707
120382
120520
119723
116592
119994
119395
118617
120483
118583
118607
118632
118698
109266
118804
118954
119137
119142
119217
116465
116490
119303
116976
119668
119715
117167
119819
119845
120082
120222
120255
128755
120338
120347
117994
119925
119314
120521
119396
119485
119718
118715
118566
119223
118539
119655
119988
129758
120670
120885
118929
116845
120478
120696
120693
119099
120122
118643
120185
118956
128690
119521
119734
120008
120594
119784
118747
120707
120888
120184
118662
116379
119548
120782
119822
99939
145302
119656
119594
120873
120491
119574
140282
120661
118744
120241
120744
120769
120890
125454
120642
119942
119780
120510
120793
119049
124179
119560
120219
120108
120772
120645
119305
119905
120598
124295
118851
120738
119272
120749
120176
120632
120728
120773
120765
124718
119196
118229
120213
120906
119792
117443
118755
119947
119508
118727
118577
120723
120725
138563
119987
120841
120742
1

189584
180837
162348
180793
220100
156460
93185
96324
172495
167203
153464
188674
85276
80913
80923
85970
185183
210616
165831
156323
174336
171359
89791
181786
96259
104438
130623
217630
215129
196216
80881
119746
120755
184894
207998
154443
116661
208003
173910
80365
98143
100438
102728
107899
110916
189047
95977
105466
92638
103767
82252
94980
86979
97322
89841
99582
84522
86129
89826
86143
90849
90190
98503
98502
91630
100403
93894
88172
87004
94631
82694
89530
99141
92563
91635
87262
105428
94612
105316
102951
100046
94882
92710
93011
187078
179074
182295
145503
162711
143422
99726
82009
93605
102592
82220
82945
90728
204175
162650
120913
186253
165384
120630
183505
187393
131704
177971
200669
97100
106469
89118
101846
99797
87727
89604
94792
87175
99800
102065
159382
219854
175142
179116
212974
87277
120903
200530
168794
120894
163676
161081
104299
82622
103241
105813
80310
205461
144528
171865
174480
200550
164052
186566
156757
206226
195234
80472
106627
95863
110823
138703
9592

314166
280590
258273
265591
107438
107443
104897
102573
111653
105885
308506
120912
289408
263725
305396
253556
257044
244094
89798
220506
271367
267626
243585
295178
276033
303353
271219
246500
88286
295427
286106
287717
279113
309377
295701
274309
303356
283288
285869
327036
93780
180052
300532
256276
265459
261289
258153
281322
280424
283991
268219
267563
88885
90799
82186
102059
94597
109368
90966
94025
83336
295254
187512
235553
103125
275688
269095
283026
219756
84133
87507
303714
265808
293662
177746
280760
106315
99128
103924
99512
101821
89274
107212
110557
91544
100666
103003
108258
171685
98354
308208
280460
240510
280380
245171
272730
274812
326306
256415
290095
330136
285487
245479
179098
318068
284494
298388
289765
289668
280491
246677
164810
265307
271259
101898
95649
95895
103069
98575
119068
129908
86373
103977
82362
273607
99581
84049
120536
276816
297037
211465
292644
283084
283139
310793
328962
272338
291502
186719
267248
227984
298130
298744
292508
252480
293832
14

284978
287839
142032
114720
259685
89789
97424
94048
300274
295743
102035
192023
102116
105121
104954
235327
86637
88322
118705
100260
209077
87003
91083
129634
102800
92906
120199
95756
105191
105159
343168
282593
108327
103872
105103
107247
93137
122143
285728
102631
99612
157913
99399
90888
101252
88925
93913
104697
280674
181810
189553
243508
104346
86489
102820
92494
85478
101373
111666
100258
108366
119630
106307
iter 5000
212879
87957
107843
100604
100112
105706
83090
298148
319262
350028
367085
367790
323298
332375
304141
297284
296572
356634
327162
391225
374900
103976
101746
85461
106877
93175
314431
110857
89160
87337
146879
105150
94331
99166
120423
100129
92752
139388
119561
98528
390521
95037
196069
100133
204137
91209
289944
104073
327437
319531
364725
362227
377752
381707
348593
332280
338512
352277
331525
96953
220343
272045
342213
339840
86251
99851
168122
104810
110944
230512
250468
87910
104573
165086
282552
299854
86873
93693
111205
238321
361596
309614
342956
3575

...

3484800
2548208
1334536
1854580
2473624
1339098
1158308
91173
87385
90079
2205501
1568343
90184
107930
95384
3576072
4007554
91165
93315
91387
119598
94592
186910
2088865
2977110
2523756
2608030
88025
99212
89647
86161
92217
1758770
2872750
1969062
460012
90564
97528
82956
150179
98411
89820
1404386
99138
92210
84533
94916
83628
81611
84010
115656
2493486
1479183
4299972
2555302
88891
88195
91909
4324274
82656
2361846
99463
85679
103213
89305
89997
124869
4083770
72173
95325
95381
91789
88800
97905
2404425
181012
379487
3449252
838164
3033948
4073952
3983738
426148
3141866
395474
2551492
1661820
316788
3609402
318181
379265
87418
89304
91631
94941
132408
91969
87837
89912
88255
94714
91727
88207
97994
80809
94946
103917
2962876
87648
87830
91534
89538
103854
85547
84241
86985
88848
87333
92199
2876428
85588
96239
93778
82812
91682
86050
192573
1648204
91927
89974
88775
93201
98557
85546
84848
2404555
80465
156413
1964418
3560546
2376218
2936470
898897
3112966
3481232
3399112
113638
108

2385195
1618421
2070883
1090680
871438
2405372
1795050
1507351
485061
2027091
4135218
194119
2448028
495750
2359814
1258154
449057
3687186
1337040
1366687
1837574
98165
1823096
2870648
118686
113535
168787
2228000
2245151
88200
91106
4692656
2027255
303151
487176
1722638
3108584
259017
208037
98089
2184207
2180168
993755
1093374
2113809
312612
304636
1541123
2287663
1663673
2231138
887143
2558318
370987
2268016
104149
3270108
2828954
91244
2466212
1560720
2091343
1615902
1675163
116224
104365
2317171
138444
1262406
2704578
2585548
131349
107702
156859
1612782
371835
218121
117794
3576038
3267194
1703231
1288589
1727816
85366
97707
107046
2010976
3301196
82134
884038
3040964
4274822
1137470
487037
476293
1729615
122247
1201135
117077
2388759
118827
1995477
1592265
1825806
2146834
981286
1543566
246834
1307897
1243939
1942972
3213222
411377
480827
768131
1303232
847759
1638355
3470838
2040367
102882
1242544
313608
1951176
1703919
2011300
1210355
4324302
818602
1652361
94827
2338062
13655

138393
299988
81053
2137376
88149
2855724
4355574
4995590
3899932
2825230
2109127
3379814
2238470
3745906
4085696
1067084
3526462
2297031
4079142
2611390
2914740
83936
843868
2297906
3286052
240641
3140724
2090594
1826550
3919598
1587427
86838
1008010
1612319
4900018
1351099
3597400
4420110
2206082
1615885
2189418
2883236
1959325
3169780
1388402
4156924
2537390
2236160
2125685
2559458
2570414
3480556
2996684
1614950
2385138
1242755
1471236
1611840
2251662
1275861
1937388
1156444
2471940
973847
2828840
1129415
102351
473267
1238304
2024506
287624
2824852
444626
2825120
2550112
3922810
3118452
1236370
425414
416853
341334
455400
914845
1465499
1372692
3560580
1305730
1572168
3529656
2049589
3272570
1051904
4504040
3118874
1254322
3277624
3397948
1943014
2739338
4338434
3293462
1558952
107415
2343266
1766044
2245003
3877296
865560
1592287
2910508
1333015
2049636
3862750
3164256
283453
97971
1817771
4024210
1275724
2109141
2822578
3208802
3355510
1772261
3735554
3181776
4163020
2788512
466

3434010
3520290
2318701
3024324
3658772
2401099
1874513
430420
5083702
2073693
1552639
119881
2216088
104298
1543883
3758280
1078927
3400060
326576
332766
331479
326600
294662
3148552
4935334
4441422
2904798
3762974
370295
923824
365653
226919
3130776
4701426
132082
455583
1152403
275039
454953
1241018
1503149
104787
339918
117457
115631
119838
178821
2501366
104267
1244093
82580
1107397
1278153
1068958
3832096
1580021
819810
1023345
1540068
3246908
1851006
82813
4920274
1480658
1740661
805184
381716
810082
163871
1626135
1527626
268677
325725
81340
1601458
2505856
4592572
2250426
2106331
2571782
3252998
3543258
3113456
146664
1276121
99274
496799
400905
1700810
1262414
1376460
2740710
2756412
1094162
2368182
1935201
2385006
3233704
5278466
1525344
2848252
340178
366276
345043
371922
364621
371775
347416
369516
362771
373856
3533916
84074
267736
358480
306877
422236
430328
408976
391274
416282
352314
409724
364604
411815
464022
1379092
4188192
4698684
3469284
3399484
2271563
444781
456

864331
399485
3949660
1105729
896816
1878841
101857
2452386
5724616
176892
242546
2924336
4558042
1398028
2025511
4494956
3960412
113633
1533089
5051278
2201532
3267734
2822584
4057438
3780208
4386134
1641388
4083572
795176
1533395
3723820
5377658
3791216
2076372
5192124
4096620
3065204
110851
2432918
5278596
92855
2113792
4247618
4790268
2674430
995868
97159
2630898
4073890
1533058
1223082
3568002
3257550
1289401
468771
3703836
871026
4151192
88883
2007506
3974790
3957986
775440
2724532
133197
2395146
4483220
2554842
3608646
4230700
4353376
498351
104523
3187378
5105736
4061010
105604
2544074
3982118
1062161
1764614
4016226
1290400
101931
95759
5512872
292513
470702
1489889
1848771
2009432
5275892
5480528
2634414
1808304
1709173
3315386
3538328
1845842
2317379
2325517
5246328
2066128
86476
3898028
1703957
1910605
88784
5432446
5481404
861350
106175
365130
4196776
2263814
2246724
1388903
1629715
1292703
1907761
3288930
4456850
86349
83976
2516274
4405860
1570964
4211044
1388369
1683048

In [159]:
df['num_faces'] = num_faces

df['brightness'] = brightness
df['saturation'] = saturation
df['hue'] = hue
df['brightness_sd'] = brightness_sd
df['saturation_sd '] = saturation_sd
df['hue_sd'] = hue_sd

df['blue'] = blue
df['blue_sd'] = blue_sd
df['green'] = green
df['green_sd'] = green_sd
df['red'] = red
df['red_sd '] = red_sd

In [160]:
df.shape

(20335, 46)

In [161]:
df.to_csv('../prprocessed.csv', index=False)

In [163]:
df = df.drop(columns = ['index'])